In [4]:
from ultralytics import YOLO
import os
import torch

def detect_ingredients(image_path):
    device = ""
    if torch.cuda.is_available():
        device = "cuda"  # Use GPU if available
    else:
        device = "cpu"  # Fallback to CPU
    # Load trained model
    model = YOLO(".\models\YOLO\last.pt").to(device)
    # Perform inference
    results = model(image_path)
    
    # Get the image name
    image_name = os.path.basename(image_path)

    # Extract ingredients and remove item counts
    ingredients = []
    for result in results:
        # Save the image with bounding boxes
        result.save(filename=f"./results/{image_name}")  # Save the image to the results directory
        
        result.show()  # display to screen
        for *xyxy, conf, cls in result.boxes.data:  # iterate over detections
            ingredient_name = model.names[int(cls)]  # get ingredient name from class ID
            # Remove any digits and leading/trailing spaces from the ingredient name
            ingredient_name = ''.join([i for i in ingredient_name if not i.isdigit()]).strip()
            # Only add ingredient if it's not already in the list
            if ingredient_name not in ingredients:
                ingredients.append(ingredient_name)  # add to ingredients list

    # Join ingredients into a string
    ingredients_str = ', '.join(ingredients)
    ingredients_str = ingredients_str.replace('_', ' ')

    return ingredients_str  # return the string of ingredients

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Function to generate text
def generate_text(input_text):
    device = ""
    if torch.cuda.is_available():
        device = "cuda"  # Use GPU if available
    else:
        device = "cpu"  # Fallback to CPU
    # Load trained model and tokenizer
    model_path = ".\models\T5"
    model_trained = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    inputs = tokenizer(input_text, return_tensors="pt", padding=True).to(device)
    outputs = model_trained.generate(**inputs, max_new_tokens=512, num_beams=2, no_repeat_ngram_size=2, repetition_penalty=1.2)  # Adjust as needed

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


In [7]:
def generate_recipe(ingredients_string):
    prompt = f"Generate recipe from these ingredients: {ingredients_string}"
    generated_text = generate_text(prompt)
    print(f"Prompt: {prompt}")
    print(f"Generated Recipe: {generated_text}")

In [6]:
image_path = "images\DSC_6121_JPG_jpg.rf.b09901df87ee20edf0cf5f758ba7bd78.jpg"

In [9]:
ingredients = detect_ingredients(image_path)
# print(ingredients)
generate_recipe(ingredients)


image 1/1 e:\Project\ml-docker-flask-api-main\app\images\DSC_6121_JPG_jpg.rf.b09901df87ee20edf0cf5f758ba7bd78.jpg: 640x448 1 bread, 1 cheese, 1 chicken, 1 chicken_breast, 1 eggs, 1 flour, 1 green_beans, 1 ground_beef, 1 milk, 1 mushrooms, 1 potato, 1 shrimp, 1 spinach, 1 tomato, 100.0ms
Speed: 0.0ms preprocess, 100.0ms inference, 231.7ms postprocess per image at shape (1, 3, 640, 448)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Prompt: Generate recipe from these ingredients: spinach, bread, ground beef, chicken, green beans, chicken breast, mushrooms, flour, milk, cheese, eggs, potato, shrimp, tomato
Generated Recipe: Title: Spinach And Chicken Soup Ingredients: spinach, bread, ground beef, chicken, green beans, poultry breast, mushrooms, flour, milk, cheese, eggs, potato, shrimp, tomato Instructions ['Preheat oven to 350 degrees F.', 'In a large bowl, combine the spinach and bread; stir in the flour and milk. Add the eggs and cook until smooth. Stir in potatoes and mushrooms. Cook for 1 hour. Remove from heat and set aside. Serve with tomato sauce. Sprinkle with salt and pepper. Cover and refrigerate for 10 minutes. Bake at 350° for 30 minutes or until golden brown. Cool for 5 minutes before serving. Let cool completely. Transfer to 375°F (180 degrees C).']


In [ ]:

import requests

url = 'http://localhost:8000/predict'
files = {'file': open('DSC_6121_JPG_jpg.rf.b09901df87ee20edf0cf5f758ba7bd78.jpg', 'rb')}  # Specify the file you want to upload

response = requests.post(url, files=files)
print(response.content)

<Response [200]>


In [3]:
print(response.content)

b'{"predicted_ingredients":"spinach, bread, ground beef, chicken, green beans, chicken breast, mushrooms, flour, milk, cheese, eggs, potato, shrimp, tomato","generated_recipe":"Title: Spinach And Chicken Soup Ingredients: spinach, bread, ground beef, chicken, green beans, poultry breast, mushrooms, flour, milk, cheese, eggs, potato, shrimp, tomato Instructions [\'Preheat oven to 350 degrees F.\', \'In a large bowl, combine the spinach and bread; stir in the flour and milk. Add the eggs and cook until smooth. Stir in potatoes and mushrooms. Cook for 1 hour. Remove from heat and set aside. Serve with tomato sauce. Sprinkle with salt and pepper. Cover and refrigerate for 10 minutes. Bake at 350\xc2\xb0 for 30 minutes or until golden brown. Cool for 5 minutes before serving. Let cool completely. Transfer to 375\xc2\xb0F (180 degrees C).\']"}'
